# Creating phase diagram using trained model

In [6]:
import torch
import numpy as np

import keras
import tensorflow as tf

from pathlib import Path

import matplotlib.pyplot as plt


In [13]:
# Old (non-relevant) toy model <- need to be replaced

# This does no longer work (I would have to save the model like this: 
#   model.save("saved_models/model_original.keras", save_format="keras"))
model = keras.models.load_model(Path("saved_models") / "skyrmion-2025-02-02_002639-a=relu,a=None,b=16,d=0,d=None,d=56,d=0.0,e=1,l=0.0,l=0.1,l=0.001,l=None,m=model5,o=SGD,s=True,s=42,s=0.0,t=1,w=0.keras", compile=False)

# model = keras.layers.TFSMLayer("saved_models/model_original", call_endpoint="serving_default")

TypeError: <class 'models.Model5'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'models', 'class_name': 'Model5', 'config': {'name': 'model5_1', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}, 'registered_name': 'Model5', 'build_config': {'input_shape': None}, 'compile_config': None}.

Exception encountered: Unable to revive model from config. When overriding the `get_config()` method, make sure that the returned config contains all items used as arguments in the  constructor to <class 'models.Model5'>, which is the default behavior. You can override this default behavior by defining a `from_config(cls, config)` class method to specify how to create an instance of Model5 from its config.

Received config={'name': 

In [ ]:
from models import Model5  # Ensure this is correctly pointing to your custom model

model = keras.models.load_model(Path("saved_models") / "skyrmion-2025-02-02_002639-a=relu,a=None,b=16,d=0,d=None,d=56,d=0.0,e=1,l=0.0,l=0.1,l=0.001,l=None,m=model5,o=SGD,s=True,s=42,s=0.0,t=1,w=0.keras", custom_objects={"Model5": Model5})

TypeError: <class 'models.Model5'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'models', 'class_name': 'Model5', 'config': {'name': 'model5_1', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}, 'registered_name': 'Model5', 'build_config': {'input_shape': None}, 'compile_config': {'optimizer': {'module': 'keras.src.backend.torch.optimizers.torch_sgd', 'class_name': 'SGD', 'config': {'name': 'SGD', 'learning_rate': 0.10000000149011612, 'weight_decay': 0.004, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'momentum': 0.9, 'nesterov': True}, 'registered_name': 'SGD'}, '

In [4]:
dataset = np.load("data/train/datasetMC.npz")

data = dataset["data"]
V = dataset["values"]

In [5]:
# preds = model.predict(data)
preds = model(data)

: 

In [9]:
data.shape

(9840, 200, 200)

In [6]:
import numpy as np
dataset=np.load("data/train/skyrmion_dataset.npz")

In [7]:
dataset["test_images"].shape

(0,)

In [7]:
keras.models.load_model("saved_models/skyrmion-2025-02-02_234724-a=relu,a=None,b=16,d=0,d=None,d=56,d=0.0,e=1,f=8,l=0.0,l=0.1,l=0.001,l=None,m=model5,o=SGD,p=max,s=True,s=42,s=0.0,t=1,w=0.keras")

/home/rothals/dev/school/diploma_thesis/dev/venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<Model5 name=model5_1, built=True>

In [1]:
# from model_serialized import Model5, MCDropout, MCSpatialDropout2D
import argparse
import keras
from models import Model5

2025-02-02 13:26:18.971263: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-02 13:26:18.971682: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-02 13:26:18.973772: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-02 13:26:18.979798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738499178.990452   96982 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738499178.99

In [9]:
parser = argparse.ArgumentParser()
parser.add_argument("--activation", default="relu", type=str, help="Activation type (see keras.activations to see all options)")
parser.add_argument("--augment", default=None, type=str, choices=["cutmix", "mixup"], help="Augmentation type")
parser.add_argument("--batch_size", default=16, type=int, help="Batch size.")
parser.add_argument("--dataloader_workers", default=0, type=int, help="Dataloader workers")
parser.add_argument("--decay", default=None, type=str, choices=["linear", "exponential", "cosine", "piecewise"], help="Decay type")
parser.add_argument("--depth", default=56, type=int, help="Model depth")
parser.add_argument("--dropout", default=0., type=float, help="Dropout")
parser.add_argument("--epochs", default=3, type=int, help="Number of epochs.")
parser.add_argument("--filters", default=8, type=int, help="Number of filters in the first convolutional layer")
parser.add_argument("--label_smoothing", default=0., type=float, help="Label smoothing")
parser.add_argument("--learning_rate", default=0.1, type=float, help="Learning rate")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate")
parser.add_argument("--logdir_suffix", default=None, type=str, help="If specified, logs will be saved in 'logs_{logdir_suffix}/' directory")
parser.add_argument("--model", default="model5", type=str, choices=["model5", "resnet"], help="Model of choice")
parser.add_argument("--optimizer", default="SGD", type=str, choices=["SGD", "Adam"], help="Optimizer type")
parser.add_argument("--pooling", default="max", type=str, choices=["max", "average"], help="Pooling type")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--save_model", default=True, type=bool, help="If True, trained model will be saved in'saved_models/' directory")
parser.add_argument("--stochastic_depth", default=0., type=float, help="Stochastic depth")
parser.add_argument("--threads", default=1, type=int, help="Maximum number of threads to use")
parser.add_argument("--weight_decay", default=0.004, type=float, help="Weight decay")
parser.add_argument("--width", default=1, type=int, help="Model width")
args = parser.parse_args([] if "__file__" not in globals() else None)

In [10]:
model = Model5(args)

In [4]:
model.save("plswork.keras")

In [5]:
loaded_model = keras.models.load_model("plswork.keras")#, custom_objects={"MCDropout": MCDropout, "MCSpatialDropout2D": MCSpatialDropout2D})

TypeError: <class 'models.Model5'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'models', 'class_name': 'Model5', 'config': {'name': 'model5', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}, 'registered_name': 'Model5', 'build_config': {'input_shape': None}}.

Exception encountered: vars() argument must have __dict__ attribute

In [56]:
vars(args)

{'activation': 'relu',
 'augment': None,
 'batch_size': 16,
 'dataloader_workers': 0,
 'decay': None,
 'depth': 56,
 'dropout': 0.0,
 'epochs': 3,
 'label_smoothing': 0.0,
 'learning_rate': 0.1,
 'learning_rate_final': 0.001,
 'logdir_suffix': None,
 'model': 'model5',
 'optimizer': 'SGD',
 'seed': 42,
 'save_model': False,
 'stochastic_depth': 0.0,
 'threads': 1,
 'weight_decay': 0.004,
 'width': 1}